# Pose2

A `Pose2` represents a robot's location and orientation in 2D space; it is a $\mathcal{SE}(2)$ manifold, so it consists of a position and a rotation: $(x, y,\theta)$. Its 3-dimensional analog is a `Pose3`. It is included in the top-level `gtsam` package.

<a href="https://colab.research.google.com/github/p-zach/myst-test/blob/main/Pose2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [13]:
import gtsam
from gtsam import Pose2, Point2, Rot2
import numpy as np

## Initialization and properties

A `Pose2` can be initialized with no arguments, which yields the identity pose, or it can be constructed with a position and rotation.

In [14]:
# Identity pose
p1 = Pose2()
print(p1)

R = Rot2.fromDegrees(90)
# Point2 is not an object, it is a utility function that creates a 2d float np.array
t = Point2(1, 2) # or (1, 2) or [1, 2] or np.array([1, 2])
# Pose at (1, 2) and facing north
p2 = Pose2(R, t)
print(p2)

(0, 0, 0)

(1, 2, 1.5708)



The pose's properties can be accessed using the appropriate members: `x()`, `y()`, `translation()` (which returns a two-element `np.array` representing `x, y`), `theta()`, `rotation()` (which returns a `Rot2`), and `matrix()`.

The `matrix()` function returns the pose's rotation and translation in the following form:

$ T = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix} = \begin{bmatrix} \cos\theta & -\sin\theta & x \\ \sin\theta & \cos\theta & y \\ 0 & 0 & 1 \end{bmatrix} $

In [15]:
print(f"Location: ({p2.x()}, {p2.y()}); also accessible with translation(): {p2.translation()}")

# .rotation() returns a Rot2 object; the float value can be accessed with .theta()
print(f"Rotation: {p2.theta()}; also accessible with rotation(): {p2.rotation().theta()}")

print(f"Position-rotation 3x3 matrix:\n{p2.matrix()}")

Location: (1.0, 2.0); also accessible with translation(): [1. 2.]
Rotation: 1.5707963267948966; also accessible with rotation(): 1.5707963267948966
Position-rotation 3x3 matrix:
[[ 6.123234e-17 -1.000000e+00  1.000000e+00]
 [ 1.000000e+00  6.123234e-17  2.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]


## Operations
### Basic operations
Points in the global space can be transformed to and from the local space of the `Pose2` using `transformTo` and `transformFrom`.

In [16]:
import math

global_point = Point2(5, 5)
origin = Pose2(Rot2.fromAngle(math.pi), Point2(1, 1))

# For a point at (1, 1) that is rotated 180 degrees, a point at (5, 5) in global
# space is at (-4, -4) in local space.
transformed = origin.transformTo(global_point)
print(f"{global_point} transformed by {origin} into local space -> {transformed}")

reversed = origin.transformFrom(transformed)
print(f"{transformed} transformed by {origin} into global space -> {reversed}")

[5. 5.] transformed by (1, 1, 3.14159)
 into local space -> [-4. -4.]
[-4. -4.] transformed by (1, 1, 3.14159)
 into global space -> [5. 5.]


Bearings (angles) and ranges (distances) can be calculated to points using the associated functions `bearing` and `range`.

In [17]:
p1 = Pose2(Rot2.fromDegrees(90), Point2(-3, -3))
point1 = Point2(-2, -3)
print(f"Bearing from {p1} to {point1}: {p1.bearing(point1).theta()}")

p2 = Pose2(Rot2.fromDegrees(-45), Point2(1, 1))
p3 = Pose2(Rot2.fromDegrees(180), Point2(0, 2))
print(f"Bearing from {p2} to {p3.translation()}: {p2.bearing(p3.translation()).theta()}")

p4 = Pose2(Rot2.fromDegrees(-90), Point2(4, 0))
point2 = Point2(0, 3)
print(f"Range from {p4} to {point2}: {p4.range(point2)}")

Bearing from (-3, -3, 1.5708)
 to [-2. -3.]: -1.5707963267948966
Bearing from (1, 1, -0.785398)
 to [0. 2.]: 3.141592653589793
Range from (4, 0, -1.5708)
 to [0. 3.]: 5.0


### Group operations
A `Pose2` implements the group operations `identity`, `inverse`, and `compose`. For more information on groups and their use here, see [GTSAM concepts](https://gtsam.org/notes/GTSAM-Concepts.html).

#### Identity

The `Pose2` identity is $(0, 0, 0)$.

#### Inverse

The inverse of a pose represents the transformation that undoes the pose. In other words, if you have a pose $T$ that moves from frame A to frame B, its inverse $T^{-1}$ moves from frame B back to frame A. The equation to compute the inverse is as follows:

$T^{-1} = (-x \cos\theta - y \sin\theta, x \sin\theta - y \cos\theta, -\theta)$

#### Composition

The composition of two `Pose2` objects follows the rules of $\mathcal{SE}(2)$ transformation.

Given two poses:
- Pose A: $T_A = (x_A, y_A, \theta_A)$
- Pose B: $T_B = (x_B, y_B, \theta_B)$

The composition of these two poses $T_C = T_A \cdot T_B$ results in:

$x_C = x_A + \cos(\theta_A) x_B - \sin(\theta_A) y_B$

$y_C = y_A + \sin(\theta_A) x_B + \cos(\theta_A) y_B$

$\theta_C = \theta_A + \theta_B$

Therefore:

$T_C = (x_A + \cos(\theta_A) x_B - \sin(\theta_A) y_B, y_A + \sin(\theta_A) x_B + \cos(\theta_A) y_B, \theta_A + \theta_B)$

In other words:
- The rotation of Pose A is applied to the translation of Pose B before adding it.
- The final rotation is just the sum of the two rotations.

In [18]:
print(f"Pose2 identity: {Pose2.Identity()}")

p1 = Pose2(0, Point2(-5, 2))
print(f"Inverse of {p1} -> {p1.inverse()}")

p2 = Pose2(Rot2.fromDegrees(90), Point2(4, 4))
print(f"Inverse of {p2} -> {p2.inverse()}")

print(f"Composition: {p1} * {p2} -> {p1 * p2}")
print(f"Composition is not commutative: {p2} * {p1} = {p2 * p1}")

Pose2 identity: (0, 0, 0)

Inverse of (-5, 2, 0)
 -> (5, -2, -0)

Inverse of (4, 4, 1.5708)
 -> (-4, 4, -1.5708)

Composition: (-5, 2, 0)
 * (4, 4, 1.5708)
 -> (-1, 6, 1.5708)

Composition is not commutative: (4, 4, 1.5708)
 * (-5, 2, 0)
 = (2, -1, 1.5708)



### Lie group operations

A `Pose2` also implements the Lie group operations for exponential mapping, log mapping, and adjoint mapping, as well as other Lie group functionalities. For more information on Lie groups and their use here, see [GTSAM concepts](https://gtsam.org/notes/GTSAM-Concepts.html).

#### Exponential mapping

The exponential map function `expmap` is used to convert a small motion, like a velocity or perturbation, in the Lie algebra (tangent space) into a `Pose2` transformation in the Lie group $\mathcal{SE}(2)$. It is used because optimization is easier in the tangent space; transformations behave like vectors there.

In tangent space, small motions are represented as:

$\xi = (\nu_x, \nu_y, \omega)$

where:
- $\nu_x, \nu_y$ are small translations in the local frame.
- $\omega$ is a small rotation.

The exponential map converts this small motion into a full pose:

$T = \exp(\xi) = \begin{cases}
    (x, y, \theta) = (\nu_x, \nu_y, \omega) & \text{if } \omega = 0 \\
    \left( \frac{\sin\omega}{\omega} \nu_x - \frac{1 - \cos\omega}{\omega} \nu_y, \frac{1 - \cos\omega}{\omega} \nu_x + \frac{\sin\omega}{\omega} \nu_y, \omega \right) & \text{otherwise}
\end{cases}$

This accounts for rotational effects when mapping from the tangent space back to $\mathcal{SE}(2)$.

#### Log mapping

The log map function `logmap` is used to convert a transformation in $\mathcal{SE}(2)$ (such as a `Pose2`) into a vector in tangent space. It can be used to convert a pose to its small motion representation or compute the difference between two poses.

For a pose $T = (x,y,\theta)$, `logmap` finds the equivalent motion in tangent space:

$\log(T) = \left( \begin{array}{c} V^{-1} \cdot t \\ \theta \\ \end{array} \right) = \xi = (\nu_x, \nu_y, \omega)$

where

$V^{-1} = \frac{1}{A^2+B^2} \left( \begin{array}{cc} A & B \\ -B & A \end{array} \right),\\ A = \frac{\sin(\theta)}{\theta},\\ B = \frac{1 - \cos(\theta)}{\theta},\\ t = (x, y)$

#### Adjoint mapping

TODO

In [69]:
# Pose2.Expmap(...) and Pose2.Logmap(...) apply their
# respective functions to the argument at identity.

pose = Pose2(Rot2.fromDegrees(135), Point2(10, 20))
xi = gtsam.Point3(10, 20, Rot2.fromDegrees(135).theta())

# what is logmap() for?
# what do I do with adjointMap()? what does it do that retract() doesn't?

simple_pose = Pose2(Rot2.fromDegrees(90), Point2(1, 0))

# twist = gtsam.Point3(.5, .5, Rot2.fromDegrees(90).theta())
twist = gtsam.Point3(.5, .5, 0)

adj_twist = simple_pose.AdjointMap() @ twist

print(adj_twist)
print(Pose2.Expmap(twist))

# Which one of these is correct?
print(simple_pose.retract(twist))
print(simple_pose.retract(adj_twist))


# print(pose.retract(twist))
# print(Pose2.Expmap(twist))
# print(pose.adjointMap_(twist))

[-0.5  0.5  0. ]
(0.5, 0.5, 0)

(0.5, 0.5, 1.5708)

(0.5, -0.5, 1.5708)



## Example: SLAM
`Pose2` can be used as the basis to perform simultaneous localization and mapping (SLAM). First, we must initialize the [factor graphs](insert link to factor graph docs)...


In [20]:
# SLAM